import time
plc=1
time.sleep(3000*plc)         #run after 1 houre = 3600 seconds

In [1]:
import os
import glob
import scipy
import random
import bisect
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import signal
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from preprocess import *                         #ماژول استخراج پنجره ها
from data import *                          #ماژول محلی ورود داده ها
from augment import *                            #ماژول های داده افزایی
from lstm_cnn import *                           #ماژول های داده افزایی با lstm_cnn
import augment
import importlib
importlib.reload(augment)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,recall_score,precision_score
from tensorflow.keras import callbacks
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D,Conv1D,Dropout,MaxPooling1D,MaxPooling2D,Flatten,Dense
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
cwd = os.getcwd() #
fullscrn()

<h1><center><div style="direction:rtl;font-family:B Nazanin">Importing Data</div></center></h1>

In [2]:
rate=480
cls_num=4
btch=80
for i in range (1,cls_num+1):
    vars()['ecg'+str(i)],vars()['ecg_tst'+str(i)]=EthanolLevel(i)

os.chdir(cwd)
clses_lens=np.array([])
i=0                               #جمع آوری داده ها و چاپ تعداد نمونه ی هر کلاس
ecg=np.array(ecg1)
print(1,len(vars()['ecg'+str(1)]), end='\t')
clses_lens=np.append(clses_lens,len(vars()['ecg'+str(1)]))
for i in range (2,cls_num+1):
    ecg=np.concatenate((ecg,vars()['ecg'+str(i)]),axis=0)
    clses_lens=np.append(clses_lens,len(vars()['ecg'+str(i)]))
    print(i,len(vars()['ecg'+str(i)]), end='\t')

mx_sig=max(clses_lens)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


first_train_shape= (504, 1752)
first_test_shape= (500, 1752)
classes_quantity= 4
tr_lbls=	 {1, 2, 3, 4}
Count_labels= [126 126 126 126]
max(train_feature_Altitude)= 4.0
min(train_feature_Altitude)= -0.99315624
first_train_sample=
 [ 1.         -0.94066444 -0.94086231 ... -0.9264013  -0.92721665
 -0.92742172]
1 126	2 126	3 126	4 126	

In [3]:
np.shape(ecg)

(504, 481)

In [4]:
ecg[:,-1]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2.

<h2><center><div style="direction:rtl;font-family:B Nazanin">Base Train windows</div></center></h1>

In [5]:
smpl_rte=480                                              # در ماژول ها نیز همین مقدار ثبت شده
i=0
windws=np.array([])

for cls in range (1,cls_num+1):                                 #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws'+str(cls)]=np.array(vars()['ecg'+str(cls)])  
    
print("each class and its windows =")

cls_wndws=np.array([])
for cls in range (1,cls_num+1):                                # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')

mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max =', mx_wndws)

each class and its windows =
1 126	2 126	3 126	4 126	
 max = 126


<h1><center><div style="direction:rtl;font-family:B Nazanin">Data Augmentation</div></center></h1>

In [6]:
smpl_rte=len(wndws1[0])                                        # در ماژول ها نیز همین مقدار ثبت شده

#add augmented data to base data
xtrain=np.empty((0,len(wndws1[0])), float) 
for i in range (1,cls_num+1):   
    xtrain=np.append(xtrain,vars()['wndws'+str(i)],axis=0)
    print('Up to class ', i, 'train shape = ', np.shape(xtrain))

Up to class  1 train shape =  (126, 481)
Up to class  2 train shape =  (252, 481)
Up to class  3 train shape =  (378, 481)
Up to class  4 train shape =  (504, 481)


In [7]:
print(np.bincount(np.int16(xtrain[:,-1])))    #تعداد پنجره در هر کلاس از 0 تا 17

[  0 126 126 126 126]


In [8]:
xtrain[:,-1]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2.

In [9]:
mx_aug=np.max(xtrain[:,:-1])
mx=np.max(ecg[:,:-1])
mn_aug=np.min(xtrain[:,:-1])
mn=np.min(ecg[:,:-1])

#for i in range (1,cls_num+1):             #نرمالسازی داده های افزایشی
xtrain[:,:-1]= 2*(xtrain[:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1

In [10]:
print('min trn =', np.min(xtrain[:,:-1]))
print('max trn =', np.max(xtrain[:,:-1]))

min trn = -1.0
max trn = 1.0


<h3><center><div style="direction:rtl;font-family:B Nazanin">Test Windows</div></center></h1>

In [11]:
wndws_test=np.empty((0,len(wndws1[0])), float)
i=0
windws=np.array([])
for cls in range (1,cls_num+1):                                 #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws_tst'+str(cls)]=np.empty((0,len(wndws1[0])), float)
    vars()['wndws_tst'+str(cls)]=np.append(vars()['wndws_tst'+str(cls)],vars()['ecg_tst'+str(cls)],axis=0)   

In [12]:
#print(np.max(wndws_tst))
for i in range (1,cls_num+1):                # Normalization test windows
    cls=i
    print('\n cls', i, ' >> ')
    print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
    print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))

    vars()['wndws_tst'+str(i)][:,:-1]= 2*(vars()['wndws_tst'+str(i)][:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1
    
    print('after normalizing >>')
    print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
    print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
    
print("each class and its windows =")                    #تجمیع کلاس های تست
cls_wndws=np.array([])
for cls in range (1,cls_num+1):                                # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws_tst'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')
    wndws_test=np.append(wndws_test,vars()['wndws_tst'+str(cls)],axis=0)
    
mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max instance in classes of test windows =', mx_wndws)


 cls 1  >> 
max magnitude class 1  =  2.0066322303209314
min magnitude class 1  =  -0.9887515177465563
after normalizing >>
max magnitude class 1  =  0.9577038578449146
min magnitude class 1  =  -0.9973014360269996

 cls 2  >> 
max magnitude class 2  =  2.0087651656600385
min magnitude class 2  =  -0.9839751932466482
after normalizing >>
max magnitude class 2  =  0.9590959665796073
min magnitude class 2  =  -0.9941840592673123

 cls 3  >> 
max magnitude class 3  =  2.011338228512399
min magnitude class 3  =  -0.9923098777256709
after normalizing >>
max magnitude class 3  =  0.9607753345407493
min magnitude class 3  =  -0.9996238805555709

 cls 4  >> 
max magnitude class 4  =  2.004223552511414
min magnitude class 4  =  -0.9939171120828527
after normalizing >>
max magnitude class 4  =  0.9561317795187649
min magnitude class 4  =  -1.0006728785942398
each class and its windows =
1 126	2 124	3 126	4 124	
 max instance in classes of test windows = 126


In [13]:
print('min tst =', np.min(wndws_test[:,:-1]))
print('max tst =', np.max(wndws_test[:,:-1]))

min tst = -1.0006728785942398
max tst = 0.9607753345407493


In [14]:
len(wndws_test)

500

In [15]:
np.shape(wndws_test)

(500, 481)

In [16]:
Xtest=wndws_test[:,:-1]
ytest=np.int16(wndws_test[:,-1])

In [17]:
fullprint(ytest)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,

<h3><center><div style="direction:rtl;font-family:B Nazanin">Train Windows</div></center></h1>

In [18]:
#ytraint=np.int32(np.array([ytrain]).T)
#trainx=np.concatenate((Xtrain, ytraint), axis=1)
trainx=np.random.permutation(xtrain)
Xtrain=np.array(trainx[:,:-1])
ytrain=np.int16(trainx[:,-1])
'''
ytestt=np.int32(np.array([ytest]).T)
testx=np.concatenate((Xtest, ytestt), axis=1)
testx=np.random.permutation(testx)
Xtest=np.int32(testx[:,:-1])
ytest=np.int32(testx[:,-1])'''

'\nytestt=np.int32(np.array([ytest]).T)\ntestx=np.concatenate((Xtest, ytestt), axis=1)\ntestx=np.random.permutation(testx)\nXtest=np.int32(testx[:,:-1])\nytest=np.int32(testx[:,-1])'

In [19]:
np.shape(wndws1)

(126, 481)

<h3><center><div style="direction:rtl;font-family:B Nazanin">Validation Windows</div></center></h1>

In [20]:
Xvalid=np.array(Xtrain)
yvalid=np.array(ytrain)

<h1><center><div style="direction:rtl;font-family:B Nazanin">CNN And UnBalanced RAW Data</div></center></h1>
<h1><center><div style="direction:rtl;font-family:Arial">Cross Entropy Loss Function</div></center></h1>

In [21]:
X_train=np.array(Xtrain)
y_train=np.array(ytrain)
X_test=np.array(Xtest)
y_test=np.array(ytest)
X_valid=np.array(Xtrain)
y_valid=np.array(ytrain)
print('X_train => ', X_train.shape)
print('y_train => ', y_train.shape)
print('X_test  => ', X_test.shape)
print('y_test  => ', y_test.shape)
print('X_valid  => ', X_valid.shape)
print('y_valid  => ', y_valid.shape)

X_train =>  (504, 480)
y_train =>  (504,)
X_test  =>  (500, 480)
y_test  =>  (500,)
X_valid  =>  (504, 480)
y_valid  =>  (504,)


In [22]:
y_train=np.array(y_train+1)
y_test=np.array(y_test+1)
y_valid=np.array(y_valid+1)

In [23]:
y_train_cat=to_categorical(y_train)
y_valid_cat=to_categorical(y_valid)
y_test_cat=to_categorical(y_test)


X_train = np.expand_dims(X_train, axis=2)
X_valid = np.expand_dims(X_valid, axis=2)
X_test = np.expand_dims(X_test, axis=2)
#y_train=np.transpose([y_train])

In [24]:
#VGG16_1D

accuracy=0
acc_crs=np.array([])
##for i in range (10):
##    if accuracy<0.2 :

verbose, epochs, batch_size = 1, 1000, 80
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
steps_per_epoch = len(X_train)//batch_size
validation_steps = len(X_valid)//batch_size # if you have test data



model_crs = Sequential()
#model_crs.add(Conv1D(input_shape=x_train.shape[1:],filters=64,kernel_size=9,padding="same", activation="relu"))
model_crs.add(Conv1D(filters=128, kernel_size=12, strides=1, activation='relu', input_shape=(n_timesteps,n_features))) #Replaced
model_crs.add(BatchNormalization())
model_crs.add(MaxPooling1D(pool_size=2,strides=3))

model_crs.add(Conv1D(filters=32, kernel_size=7, strides=1, padding="same", activation="relu"))
model_crs.add(BatchNormalization())
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=32, kernel_size=10, strides=1, padding="same", activation="relu"))
model_crs.add(Conv1D(filters=128, kernel_size=5, strides=2, padding="same", activation="relu"))
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=256, kernel_size=15, strides=1, padding="same", activation="relu"))
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=512, kernel_size=5, strides=1, padding="same", activation="relu"))
#model_crs.add(Conv1D(filters=128, kernel_size=3, strides=1, padding="same", activation="relu"))


model_crs.add(Flatten())
model_crs.add(Dropout(0.1))
model_crs.add(Dense(units=512,activation="relu"))
#model_crs.add(Dense(units=2048,activation="relu"))
model_crs.add(Dense(n_outputs, activation='softmax'))   #Replaced here from old Model

model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
# fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy

earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 50, restore_best_weights = True)

model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])

_, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=1)

print('np.shape(X_test)=',np.shape(X_test))

print('Accuracy= ', accuracy)

#y_pred_vgg = model_crs.predict_classes(X_test)


Epoch 1/1000
7/7 [==============================] - 4s 84ms/step - loss: 2.9097 - accuracy: 0.2401 - val_loss: 1.6697 - val_accuracy: 0.2500
Epoch 2/1000
7/7 [==============================] - 0s 24ms/step - loss: 1.5083 - accuracy: 0.2381 - val_loss: 1.5804 - val_accuracy: 0.2500
Epoch 3/1000
7/7 [==============================] - 0s 24ms/step - loss: 1.4422 - accuracy: 0.2341 - val_loss: 1.5684 - val_accuracy: 0.2500
Epoch 4/1000
7/7 [==============================] - 0s 24ms/step - loss: 1.4048 - accuracy: 0.2817 - val_loss: 1.4823 - val_accuracy: 0.2500
Epoch 5/1000
7/7 [==============================] - 0s 24ms/step - loss: 1.4145 - accuracy: 0.2421 - val_loss: 1.5154 - val_accuracy: 0.2500
Epoch 6/1000
7/7 [==============================] - 0s 24ms/step - loss: 1.4094 - accuracy: 0.2639 - val_loss: 1.5511 - val_accuracy: 0.2500
Epoch 7/1000
7/7 [==============================] - 0s 24ms/step - loss: 1.4301 - accuracy: 0.2361 - val_loss: 1.4480 - val_accuracy: 0.2500
Epoch 8/1000


Epoch 59/1000
7/7 [==============================] - 0s 24ms/step - loss: 1.0349 - accuracy: 0.5179 - val_loss: 1.2139 - val_accuracy: 0.5099
Epoch 60/1000
7/7 [==============================] - 0s 24ms/step - loss: 0.9207 - accuracy: 0.5556 - val_loss: 1.2480 - val_accuracy: 0.4524
Epoch 61/1000
7/7 [==============================] - 0s 24ms/step - loss: 0.9936 - accuracy: 0.6091 - val_loss: 1.1457 - val_accuracy: 0.4980
Epoch 62/1000
7/7 [==============================] - 0s 24ms/step - loss: 0.9107 - accuracy: 0.5377 - val_loss: 1.2326 - val_accuracy: 0.4286
Epoch 63/1000
7/7 [==============================] - 0s 25ms/step - loss: 0.9804 - accuracy: 0.5575 - val_loss: 1.2092 - val_accuracy: 0.4127
Epoch 64/1000
7/7 [==============================] - 0s 24ms/step - loss: 1.0085 - accuracy: 0.6131 - val_loss: 1.2754 - val_accuracy: 0.3611
Epoch 65/1000
7/7 [==============================] - 0s 26ms/step - loss: 0.9545 - accuracy: 0.5952 - val_loss: 0.9715 - val_accuracy: 0.5813
Epoch 

7/7 [==============================] - 0s 25ms/step - loss: 0.2405 - accuracy: 0.9067 - val_loss: 0.1689 - val_accuracy: 0.9504
Epoch 117/1000
7/7 [==============================] - 0s 24ms/step - loss: 0.3407 - accuracy: 0.8591 - val_loss: 0.5364 - val_accuracy: 0.7460
Epoch 118/1000
7/7 [==============================] - 0s 24ms/step - loss: 0.3251 - accuracy: 0.8690 - val_loss: 0.1815 - val_accuracy: 0.9345
Epoch 119/1000
7/7 [==============================] - 0s 24ms/step - loss: 0.3215 - accuracy: 0.8750 - val_loss: 0.6958 - val_accuracy: 0.7321
Epoch 120/1000
7/7 [==============================] - 0s 24ms/step - loss: 0.4139 - accuracy: 0.8413 - val_loss: 0.7029 - val_accuracy: 0.7004
Epoch 121/1000
7/7 [==============================] - 0s 24ms/step - loss: 0.4509 - accuracy: 0.8512 - val_loss: 0.2154 - val_accuracy: 0.9127
Epoch 122/1000
7/7 [==============================] - 0s 24ms/step - loss: 0.1920 - accuracy: 0.9286 - val_loss: 0.5272 - val_accuracy: 0.7758
Epoch 123/1000

7/7 [==============================] - 0s 24ms/step - loss: 0.3540 - accuracy: 0.8909 - val_loss: 0.1082 - val_accuracy: 0.9683
Epoch 174/1000
7/7 [==============================] - 0s 27ms/step - loss: 0.0388 - accuracy: 0.9921 - val_loss: 0.0154 - val_accuracy: 0.9960
Epoch 175/1000
7/7 [==============================] - 0s 26ms/step - loss: 0.0115 - accuracy: 0.9980 - val_loss: 0.0142 - val_accuracy: 0.9980
Epoch 176/1000
7/7 [==============================] - 0s 24ms/step - loss: 0.0145 - accuracy: 0.9960 - val_loss: 1.4891 - val_accuracy: 0.7480
Epoch 177/1000
7/7 [==============================] - 0s 24ms/step - loss: 0.9952 - accuracy: 0.7123 - val_loss: 0.4476 - val_accuracy: 0.7857
Epoch 178/1000
7/7 [==============================] - 0s 24ms/step - loss: 0.2184 - accuracy: 0.9266 - val_loss: 0.1251 - val_accuracy: 0.9484
Epoch 179/1000
7/7 [==============================] - 0s 24ms/step - loss: 0.0315 - accuracy: 0.9921 - val_loss: 0.2888 - val_accuracy: 0.9167
Epoch 180/1000

7/7 [==============================] - 0s 25ms/step - loss: 0.0533 - accuracy: 0.9980 - val_loss: 0.0383 - val_accuracy: 0.9861
Epoch 231/1000
7/7 [==============================] - 0s 5ms/step - loss: 0.0119 - accuracy: 1.0000
np.shape(X_test)= (500, 480, 1)
Accuracy=  1.0


In [25]:
def maxindx(ar):
    ar_cpy=np.array(ar)
    pred=np.argmax(ar,axis=1)
    for i in range (len(pred)):
        if pred[i]==0:
            ar_cpy[i,0]=-1000
    pred=np.argmax(ar_cpy,axis=1)
    return(pred)

In [26]:
predict_x=model_crs.predict(X_test)              # Function 1

y_pred_crs=maxindx(predict_x)                    # function from augment.py to remove 0 index predictions

#y_pred_crs = model_crs.predict_classes(X_test)

cm = confusion_matrix(y_test, y_pred_crs)
fullprint(cm)
acc3=accuracy_score(y_test, y_pred_crs)

print(classification_report(y_test, y_pred_crs))

#sns.heatmap(cm, annot=True)

#plt.imshow(cm)

16/16 [==============================] - 0s 8ms/step
array([[ 66,  41,  18,   1],
       [ 22,  38,  60,   4],
       [ 10,  26,  68,  22],
       [  1,   2,  13, 108]], dtype=int64)
              precision    recall  f1-score   support

           2       0.67      0.52      0.59       126
           3       0.36      0.31      0.33       124
           4       0.43      0.54      0.48       126
           5       0.80      0.87      0.83       124

    accuracy                           0.56       500
   macro avg       0.56      0.56      0.56       500
weighted avg       0.56      0.56      0.56       500



In [27]:
#VGG16_1D

accuracy=0
acc_crs=np.array([])
##for i in range (10):
##    if accuracy<0.2 :

verbose, epochs, batch_size = 1, 1000, 80
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
steps_per_epoch = len(X_train)//batch_size
validation_steps = len(X_valid)//batch_size # if you have test data



model_crs = Sequential()
#model_crs.add(Conv1D(input_shape=x_train.shape[1:],filters=64,kernel_size=9,padding="same", activation="relu"))
model_crs.add(Conv1D(filters=256, kernel_size=12, strides=1, activation='relu', input_shape=(n_timesteps,n_features))) #Replaced
model_crs.add(BatchNormalization())
model_crs.add(MaxPooling1D(pool_size=2,strides=3))

model_crs.add(Conv1D(filters=64, kernel_size=7, strides=1, padding="same", activation="relu"))
model_crs.add(BatchNormalization())
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=64, kernel_size=10, strides=1, padding="same", activation="relu"))
model_crs.add(Conv1D(filters=128, kernel_size=5, strides=2, padding="same", activation="relu"))
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=512, kernel_size=15, strides=1, padding="same", activation="relu"))
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=1024, kernel_size=5, strides=1, padding="same", activation="relu"))
#model_crs.add(Conv1D(filters=128, kernel_size=3, strides=1, padding="same", activation="relu"))


model_crs.add(Flatten())
model_crs.add(Dropout(0.1))
model_crs.add(Dense(units=512,activation="relu"))
#model_crs.add(Dense(units=2048,activation="relu"))
model_crs.add(Dense(n_outputs, activation='softmax'))   #Replaced here from old Model

model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
# fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy

earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 50, restore_best_weights = True)

model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])

_, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=1)

print('np.shape(X_test)=',np.shape(X_test))

print('Accuracy= ', accuracy)

#y_pred_vgg = model_crs.predict_classes(X_test)


Epoch 1/1000
7/7 [==============================] - 2s 99ms/step - loss: 9.0410 - accuracy: 0.1984 - val_loss: 1.7206 - val_accuracy: 0.2500
Epoch 2/1000
7/7 [==============================] - 0s 34ms/step - loss: 1.5675 - accuracy: 0.2579 - val_loss: 1.6673 - val_accuracy: 0.2500
Epoch 3/1000
7/7 [==============================] - 0s 34ms/step - loss: 1.4359 - accuracy: 0.2302 - val_loss: 1.6037 - val_accuracy: 0.2500
Epoch 4/1000
7/7 [==============================] - 0s 34ms/step - loss: 1.4170 - accuracy: 0.2460 - val_loss: 1.6012 - val_accuracy: 0.2500
Epoch 5/1000
7/7 [==============================] - 0s 33ms/step - loss: 1.4054 - accuracy: 0.2639 - val_loss: 1.5311 - val_accuracy: 0.2500
Epoch 6/1000
7/7 [==============================] - 0s 34ms/step - loss: 1.4379 - accuracy: 0.2143 - val_loss: 1.5178 - val_accuracy: 0.2500
Epoch 7/1000
7/7 [==============================] - 0s 33ms/step - loss: 1.4183 - accuracy: 0.2480 - val_loss: 1.4586 - val_accuracy: 0.2500
Epoch 8/1000


Epoch 59/1000
7/7 [==============================] - 0s 34ms/step - loss: 1.3352 - accuracy: 0.3532 - val_loss: 1.3671 - val_accuracy: 0.2996
Epoch 60/1000
7/7 [==============================] - 0s 34ms/step - loss: 1.3424 - accuracy: 0.3373 - val_loss: 1.3707 - val_accuracy: 0.3016
Epoch 61/1000
7/7 [==============================] - 0s 34ms/step - loss: 1.3326 - accuracy: 0.3611 - val_loss: 1.3787 - val_accuracy: 0.2996
Epoch 62/1000
7/7 [==============================] - 0s 33ms/step - loss: 1.3372 - accuracy: 0.3492 - val_loss: 1.3709 - val_accuracy: 0.2798
Epoch 63/1000
7/7 [==============================] - 0s 34ms/step - loss: 1.3406 - accuracy: 0.3393 - val_loss: 1.3663 - val_accuracy: 0.3512
Epoch 64/1000
7/7 [==============================] - 0s 34ms/step - loss: 1.3259 - accuracy: 0.3333 - val_loss: 1.3537 - val_accuracy: 0.3532
Epoch 65/1000
7/7 [==============================] - 0s 37ms/step - loss: 1.3055 - accuracy: 0.3571 - val_loss: 1.3189 - val_accuracy: 0.3730
Epoch 

7/7 [==============================] - 0s 34ms/step - loss: 0.8683 - accuracy: 0.6131 - val_loss: 1.2386 - val_accuracy: 0.5278
Epoch 117/1000
7/7 [==============================] - 0s 34ms/step - loss: 0.8266 - accuracy: 0.6409 - val_loss: 0.8823 - val_accuracy: 0.6310
Epoch 118/1000
7/7 [==============================] - 0s 34ms/step - loss: 0.8942 - accuracy: 0.6012 - val_loss: 0.8361 - val_accuracy: 0.6468
Epoch 119/1000
7/7 [==============================] - 0s 34ms/step - loss: 0.7307 - accuracy: 0.6885 - val_loss: 0.8636 - val_accuracy: 0.6071
Epoch 120/1000
7/7 [==============================] - 0s 34ms/step - loss: 0.7518 - accuracy: 0.6944 - val_loss: 0.9257 - val_accuracy: 0.6091
Epoch 121/1000
7/7 [==============================] - 0s 34ms/step - loss: 1.5803 - accuracy: 0.5357 - val_loss: 0.9872 - val_accuracy: 0.5694
Epoch 122/1000
7/7 [==============================] - 0s 35ms/step - loss: 0.7279 - accuracy: 0.7163 - val_loss: 1.1261 - val_accuracy: 0.5337
Epoch 123/1000

7/7 [==============================] - 0s 34ms/step - loss: 0.3889 - accuracy: 0.8651 - val_loss: 0.4487 - val_accuracy: 0.8353
Epoch 174/1000
7/7 [==============================] - 0s 34ms/step - loss: 0.1751 - accuracy: 0.9504 - val_loss: 5.0098 - val_accuracy: 0.4306
Epoch 175/1000
7/7 [==============================] - 0s 34ms/step - loss: 1.0733 - accuracy: 0.7123 - val_loss: 0.6336 - val_accuracy: 0.7798
Epoch 176/1000
7/7 [==============================] - 0s 34ms/step - loss: 0.3740 - accuracy: 0.8671 - val_loss: 0.4909 - val_accuracy: 0.8056
Epoch 177/1000
7/7 [==============================] - 0s 34ms/step - loss: 0.1905 - accuracy: 0.9167 - val_loss: 0.2227 - val_accuracy: 0.9187
Epoch 178/1000
7/7 [==============================] - 0s 37ms/step - loss: 0.3396 - accuracy: 0.8770 - val_loss: 0.0672 - val_accuracy: 0.9742
Epoch 179/1000
7/7 [==============================] - 0s 33ms/step - loss: 0.0658 - accuracy: 0.9762 - val_loss: 0.0707 - val_accuracy: 0.9643
Epoch 180/1000

7/7 [==============================] - 0s 34ms/step - loss: 0.0167 - accuracy: 0.9960 - val_loss: 0.0458 - val_accuracy: 0.9881
Epoch 231/1000
7/7 [==============================] - 0s 34ms/step - loss: 0.4098 - accuracy: 0.8631 - val_loss: 0.4032 - val_accuracy: 0.8690
Epoch 232/1000
7/7 [==============================] - 0s 34ms/step - loss: 0.6013 - accuracy: 0.8492 - val_loss: 0.5653 - val_accuracy: 0.7520
Epoch 233/1000
7/7 [==============================] - 0s 34ms/step - loss: 0.1594 - accuracy: 0.9504 - val_loss: 0.0413 - val_accuracy: 0.9861
Epoch 234/1000
7/7 [==============================] - 0s 34ms/step - loss: 0.0120 - accuracy: 1.0000 - val_loss: 0.0241 - val_accuracy: 0.9921
Epoch 235/1000
7/7 [==============================] - 0s 34ms/step - loss: 0.0088 - accuracy: 0.9980 - val_loss: 0.2870 - val_accuracy: 0.9246
Epoch 236/1000
7/7 [==============================] - 0s 34ms/step - loss: 0.5997 - accuracy: 0.8135 - val_loss: 0.5267 - val_accuracy: 0.7798
Epoch 237/1000

7/7 [==============================] - 0s 35ms/step - loss: 0.1143 - accuracy: 0.9603 - val_loss: 0.0857 - val_accuracy: 0.9663
Epoch 288/1000
7/7 [==============================] - 0s 34ms/step - loss: 0.0033 - accuracy: 1.0000 - val_loss: 0.0410 - val_accuracy: 0.9861
Epoch 289/1000
7/7 [==============================] - 0s 34ms/step - loss: 0.5631 - accuracy: 0.8135 - val_loss: 0.2747 - val_accuracy: 0.8849
Epoch 290/1000
7/7 [==============================] - 0s 34ms/step - loss: 0.2084 - accuracy: 0.9325 - val_loss: 0.5931 - val_accuracy: 0.9187
Epoch 291/1000
7/7 [==============================] - 0s 34ms/step - loss: 0.5088 - accuracy: 0.8512 - val_loss: 0.4409 - val_accuracy: 0.8869
Epoch 292/1000
7/7 [==============================] - 0s 34ms/step - loss: 0.0873 - accuracy: 0.9802 - val_loss: 0.1158 - val_accuracy: 0.9643
Epoch 293/1000
7/7 [==============================] - 0s 35ms/step - loss: 0.0317 - accuracy: 0.9921 - val_loss: 0.2589 - val_accuracy: 0.9266
Epoch 294/1000

In [28]:
predict_x=model_crs.predict(X_test)              # Function 1

y_pred_crs=maxindx(predict_x)                    # function from augment.py to remove 0 index predictions

#y_pred_crs = model_crs.predict_classes(X_test)

cm = confusion_matrix(y_test, y_pred_crs)
fullprint(cm)
acc3=accuracy_score(y_test, y_pred_crs)

print(classification_report(y_test, y_pred_crs))

#sns.heatmap(cm, annot=True)

#plt.imshow(cm)

16/16 [==============================] - 0s 9ms/step
array([[ 41,  48,  26,  11],
       [ 11,  43,  54,  16],
       [  5,  26,  58,  37],
       [  1,   5,  18, 100]], dtype=int64)
              precision    recall  f1-score   support

           2       0.71      0.33      0.45       126
           3       0.35      0.35      0.35       124
           4       0.37      0.46      0.41       126
           5       0.61      0.81      0.69       124

    accuracy                           0.48       500
   macro avg       0.51      0.48      0.48       500
weighted avg       0.51      0.48      0.47       500

